In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
import numpy as np
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/1:1_Rohan_Kaushikan/Dataset/sorted_genes.csv')

In [ ]:
gene_df = pd.read_csv('/content/drive/Shareddrives/1:1_Rohan_Kaushikan/Dataset/rohan_merged_data_rna_seq_rsem.csv')

In [ ]:
df.head(24)

,Gene,p_value
0,SNORD20,0.000000
1,SNORD114-13,0.000000
2,SNORD102,0.000000
3,DEFB116,0.000000
4,SNORD115-35,0.000000
5,DEFB112,0.000000
6,CT47A10,0.000000
7,SNORD110,0.000000
8,DAZ3,0.000000
9,SNAR-D,0.000000


In [ ]:
gene_df.head(5)

,Unnamed: 0,UBE2Q2P2,HMGB1P1,RNU12-2P,SSX9P,EZHIP,EFCAB8,SRP14P1,TRIM75P,SPATA31B1P,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,Disease_Free_Status
0,TCGA-3C-AAAU,12.9316,52.1503,0.0000,0.0000,1.7233,0.3447,4.1359,0.6893,0.0,...,931.957,129.5920,1007.780,1658.500,258.4940,1208.370,3507.25,1894.930,1180.460,1:Recurred/Progressed
1,TCGA-3C-AALI,17.3790,69.7553,1.0875,0.5438,144.1000,4.8940,1.6313,0.5438,0.0,...,965.198,59.8151,448.613,1343.120,198.4770,603.589,5504.62,1318.650,406.743,0:DiseaseFree
2,TCGA-3C-AALJ,9.2294,154.2970,0.0000,0.0000,0.0000,2.7199,2.7199,0.0000,0.0,...,2531.280,35.3581,533.998,768.812,331.8220,532.185,5458.75,942.883,509.519,0:DiseaseFree
3,TCGA-3C-AALK,11.0799,143.8640,0.4137,0.0000,0.0000,2.0687,4.5511,0.0000,0.0,...,668.597,55.0269,437.733,863.881,175.4240,607.365,5691.35,781.134,700.869,NaN
4,TCGA-4H-AAAK,14.4298,84.2128,0.4255,0.0000,0.4255,2.9787,2.9787,0.0000,0.0,...,674.468,48.9362,424.255,1049.790,14.0426,775.745,4041.70,831.915,881.702,0:DiseaseFree


In [ ]:
selected_columns = ['ATG9B', 'DUSP26', 'SIX2', 'NOTCH1', 'BCL8', 'ABCG8', 'RGS9BP', 'FSCN1', 'PTP4A3', 'NT5DC2', 'Disease_Free_Status']
top_ten_df = gene_df[selected_columns]

In [ ]:
top_ten_df.head(20)

,ATG9B,DUSP26,SIX2,NOTCH1,BCL8,ABCG8,RGS9BP,FSCN1,PTP4A3,NT5DC2,Disease_Free_Status
0,1148.6200,9.9951,120.9750,748.944,10.6844,0.6893,1.3786,627.624,1065.000,499.686,1:Recurred/Progressed
1,223.0070,1.6313,775.4210,855.356,2.7189,1.0875,14.6819,1580.210,499.184,1491.680,0:DiseaseFree
2,138.7580,2.7199,529.4650,789.665,0.9066,0.0000,2.7199,1892.110,890.299,1749.900,0:DiseaseFree
3,22.7762,4.9648,25.2379,1037.650,1.6549,0.0000,6.6198,1816.300,334.299,640.538,NaN
4,25.1277,0.8511,50.2128,1181.700,2.1277,0.4255,3.8298,1705.110,476.170,715.136,0:DiseaseFree
5,12.6294,8.7989,158.9680,847.045,0.0000,0.0000,4.6928,1783.840,496.260,1084.930,NaN
6,91.5795,191.4230,1741.6300,653.243,0.0000,0.0000,23.5356,289.749,472.280,1047.090,NaN
7,7.6180,0.4505,61.7187,3591.400,8.5595,0.0000,7.2080,2275.030,254.533,2560.640,0:DiseaseFree
8,8.8363,4.6315,17.8645,1095.690,2.9774,0.6616,2.3158,1167.810,782.731,368.046,0:DiseaseFree
9,13.2522,7.5689,25.3394,916.495,2.6327,0.0000,4.6072,1103.090,346.195,843.097,0:DiseaseFree


In [ ]:
top_ten_df.columns

Index(['ATG9B', 'DUSP26', 'SIX2', 'NOTCH1', 'BCL8', 'ABCG8', 'RGS9BP', 'FSCN1',
       'PTP4A3', 'NT5DC2', 'Disease_Free_Status'],
      dtype='object')

In [ ]:
top_ten_df.shape

(1082, 11)

In [ ]:
df_cleaned.isnull().sum()

ATG9B                  0
DUSP26                 0
SIX2                   0
NOTCH1                 0
BCL8                   0
ABCG8                  0
RGS9BP                 0
FSCN1                  0
PTP4A3                 0
NT5DC2                 0
Disease_Free_Status    0
dtype: int64

In [ ]:
df_cleaned = top_ten_df.dropna()

In [ ]:
df_cleaned['Disease_Free_Status'] = df_cleaned['Disease_Free_Status'].replace({
    '1:Recurred/Progressed': 1,
    '0:DiseaseFree': 0
})

<ipython-input-23-ea425d725b05>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Disease_Free_Status'] = df_cleaned['Disease_Free_Status'].replace({


In [ ]:
df_cleaned.head()

,ATG9B,DUSP26,SIX2,NOTCH1,BCL8,ABCG8,RGS9BP,FSCN1,PTP4A3,NT5DC2,Disease_Free_Status
0,1148.6200,9.9951,120.9750,748.944,10.6844,0.6893,1.3786,627.624,1065.000,499.686,1
1,223.0070,1.6313,775.4210,855.356,2.7189,1.0875,14.6819,1580.210,499.184,1491.680,0
2,138.7580,2.7199,529.4650,789.665,0.9066,0.0000,2.7199,1892.110,890.299,1749.900,0
4,25.1277,0.8511,50.2128,1181.700,2.1277,0.4255,3.8298,1705.110,476.170,715.136,0
7,7.6180,0.4505,61.7187,3591.400,8.5595,0.0000,7.2080,2275.030,254.533,2560.640,0


In [ ]:
df_cleaned.to_csv('/content/drive/Shareddrives/1:1_Rohan_Kaushikan/Dataset/cleaned_data.csv', index=False)

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/1:1_Rohan_Kaushikan/Dataset/cleaned_data.csv')

In [ ]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [ ]:
train_data.shape, test_data.shape, val_data.shape

((601, 11), (188, 11), (151, 11))

In [ ]:
train_data.to_csv('/content/drive/Shareddrives/1:1_Rohan_Kaushikan/Dataset/train_data.csv', index=False)

In [ ]:
test_data.to_csv('/content/drive/Shareddrives/1:1_Rohan_Kaushikan/Dataset/test_data.csv', index=False)

In [ ]:
val_data.to_csv('/content/drive/Shareddrives/1:1_Rohan_Kaushikan/Dataset/val_data.csv', index=False)